In [14]:
import time
start = time.time()

In [15]:
!nvidia-smi

'nvidia-smi' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [16]:
%pip install torch timm albumentations -q

Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from sklearn.model_selection import StratifiedKFold
import numpy as np
from tqdm import tqdm

In [18]:
# Read in the training dataset
train = pd.read_csv(f"./Train.csv")

In [19]:
train.head()

,ID,img_origin,placement,boil_nbr,pan_nbr,polygon
0,ID00rw8,D,roof,0,2,"[(2087, 2179.0), (2181, 2191.0), (2171, 2223.0..."
1,ID014O6EC7,D,roof,0,1,"[(1327, 1574.0), (1595, 1308.0), (2169, 1744.0..."
2,ID020cu0z,D,openspace,0,1,"[(2215, 1372.0), (2400, 1422.0), (2316, 1716.0..."
3,ID024YTBkLvRpQahT,D,roof,0,1,"[(1574, 526.0), (1698, 366.0), (1810, 422.0), ..."
4,ID024YTBkLvRpQahT,D,roof,0,2,"[(1402, 782.0), (1814, 854.0), (1726, 1074.0),..."


In [20]:
# Create a placement mapper
placement_mapper = train[["ID", "placement"]].drop_duplicates().set_index("ID").to_dict()
# Create a "img_origin" mapper
img_origin_mapper = train[["ID", "img_origin"]].drop_duplicates().set_index("ID").to_dict()

# Group by "ID" and sum up boil_nb, pan_nbr
train_df = train.groupby("ID").sum().reset_index()[["ID", "boil_nbr", "pan_nbr"]]

# Map img_origin and placement
train_df["img_origin"] = train_df["ID"].map(img_origin_mapper["img_origin"])
train_df["placement"] = train_df["ID"].map(placement_mapper["placement"])

# Create path column
train_df["path"] = "/kaggle/input/lacuna-solar-survey-challenge/images/" + train_df["ID"] + ".jpg"

In [21]:
train_df.head()

,ID,boil_nbr,pan_nbr,img_origin,placement,path
0,ID00rw8,0,2,D,roof,/kaggle/input/lacuna-solar-survey-challenge/im...
1,ID014O6EC7,0,1,D,roof,/kaggle/input/lacuna-solar-survey-challenge/im...
2,ID020cu0z,0,1,D,openspace,/kaggle/input/lacuna-solar-survey-challenge/im...
3,ID024YTBkLvRpQahT,0,5,D,roof,/kaggle/input/lacuna-solar-survey-challenge/im...
4,ID02vByTw8Htl,0,1,D,roof,/kaggle/input/lacuna-solar-survey-challenge/im...


In [22]:
# Stratified KFold based on multi-label targets
train_df["stratify_label"] = train_df[["boil_nbr", "pan_nbr"]].sum(axis=1)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_df["fold"] = -1
for fold, (_, valid_idx) in enumerate(skf.split(train_df, train_df["stratify_label"])):
    train_df.loc[valid_idx, "fold"] = fold

c:\Users\ATD\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [23]:
# Define Transformations
train_transforms = A.Compose([
    A.Resize(384, 384), 
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])


test_transforms = A.Compose([
    A.Resize(384, 384),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [24]:
# Custom Dataset
class SolarPanelDataset(Dataset):
    def __init__(self, dataframe, transform=None, to_train=True):
        self.dataframe = dataframe
        self.transform = transform
        self.to_train = to_train

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = cv2.imread(row["path"])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)["image"]

        if self.to_train:
          target = torch.tensor([row["boil_nbr"], row["pan_nbr"]], dtype=torch.float32)
          return image, target
        else:
          return image

In [25]:
# Prepare Dataloaders
fold = 0
train_data = train_df[train_df["fold"] != fold].reset_index(drop=True)
valid_data = train_df[train_df["fold"] == fold].reset_index(drop=True)

dataset_train = SolarPanelDataset(train_data, transform=train_transforms)
dataset_valid = SolarPanelDataset(valid_data, transform=test_transforms)

train_loader = DataLoader(dataset_train, batch_size=8, shuffle=True, num_workers=os.cpu_count())
valid_loader = DataLoader(dataset_valid, batch_size=8, shuffle=False)

In [26]:
import torch.nn as nn
import timm

# Model Definition
class EfficientNetRegressor(nn.Module):
    def __init__(self):
        super(EfficientNetRegressor, self).__init__()
        # Upgrade to EfficientNet-B3
        self.model = timm.create_model("tf_efficientnet_b5", pretrained=True)
        
        # Get the number of features from the classifier layer
        in_features = self.model.classifier.in_features
        
        # Replace the classifier with a custom regression head
        self.model.classifier = nn.Sequential(
            nn.Linear(in_features, 512),  # Reduce to a hidden layer of 512 units
            nn.ReLU(),                   # Non-linearity
            nn.Dropout(0.3),             # Dropout for regularization (30% dropout rate)
            nn.Linear(512, 2)            # Output layer for 2 regression targets (boil_nbr, pan_nbr)
        )

    def forward(self, x):
        # Forward pass through the model
        return self.model(x)

In [27]:
# Training Setup
model = EfficientNetRegressor()
criterion = nn.L1Loss()  # MAE Loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Model Saving Path
best_model_path = "/kaggle/working/best_model.pth"

model.safetensors:   0%|          | 0.00/122M [00:00<?, ?B/s]

c:\Users\ATD\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ATD\.cache\huggingface\hub\models--timm--tf_efficientnet_b5.ns_jft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [28]:
%%time
# CPU times: user 18min 28s, sys: 4min 57s, total: 23min 26s
# Wall time: 32min 26s

# Training Loop
num_epochs = 10
patience = 3
best_loss = float("inf")
counter = 0

# Check device (optional, for flexibility if GPU becomes available later)
device = torch.device("cuda")
print(f"Using device: {device}")

# Assuming model, train_loader, valid_loader, criterion, optimizer, and best_model_path are defined elsewhere
model = model.to(device)  # Move model to CPU (or other device if specified)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, targets = images.to(device), targets.to(device)  # Move data to CPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Validation Loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            images, targets = images.to(device), targets.to(device)  # Move data to CPU
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    scheduler.step(val_loss)

    val_loss /= len(valid_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")

    # Save Best Model
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), best_model_path)
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            break

Using device: cuda


AssertionError: Torch not compiled with CUDA enabled

In [29]:
# Load Best Model
model.load_state_dict(torch.load(best_model_path))
model.eval()

# Predict on Validation Set
preds = []
true_vals = []
with torch.no_grad():
    for images, targets in tqdm(valid_loader, desc="Predicting on Validation Set"):
        images = images.cuda()
        outputs = model(images).cpu().numpy()
        preds.append(outputs)
        true_vals.append(targets.numpy())
preds = np.concatenate(preds, axis=0)
true_vals = np.concatenate(true_vals, axis=0)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/best_model.pth'

In [ ]:
from sklearn.metrics import mean_absolute_error

# Evaluate using MAE
mae = mean_absolute_error(true_vals, preds)
print(f"Validation MAE: {mae:.4f}")

# Previous Validation MAE: 1.2880

In [ ]:
%%time
# CPU times: user 1min 51s, sys: 58.3 s, total: 2min 49s
# Wall time: 2min 22s

# Predict on Test Set
test_df = pd.read_csv(f"/kaggle/input/lacuna-solar-survey-challenge/Test.csv")
test_df["path"] = "/kaggle/input/lacuna-solar-survey-challenge/images/" + test_df["ID"] + ".jpg"

dataset_test = SolarPanelDataset(test_df, transform=test_transforms, to_train=False)
test_loader = DataLoader(dataset_test, batch_size=8, shuffle=False)

test_preds = []
with torch.no_grad():
    for images in tqdm(test_loader, desc="Predicting on Test Set"):
        images = images.cuda()
        outputs = model(images).cpu().numpy()
        test_preds.append(outputs)
test_preds = np.concatenate(test_preds, axis=0)

In [ ]:
# Create Sample Submission
submission = pd.DataFrame()
submission["ID"] = np.repeat(test_df["ID"].values, 2)
submission["ID"] = submission["ID"] + np.tile(["_boil", "_pan"], len(test_df))
submission["Target"] = test_preds.flatten().clip(0,1000)

# Save Submission
submission.to_csv("Submission.csv", index=False)
print("Submission saved!")

In [ ]:
end = time.time()
expected = (end - start) / 60
print(f"Expected runtime: {expected:.2f} minutes")

In [ ]:
# LB score: 1.385280187 LB rank: 23